# Lab 5 Submission: Langugae Modelling with a Sequence 2 Sequence Model
This notebbok contains the code used to convert the original code given from Machine Translation (MT) to Language Modeling (LM) and tests the trained model for  next word prediction. The results indicate that the model struggles in this area, likely due to the limited training data.
Any changes from the original codebase will be outlined throughout the notebook.

The main changes stemmed from the fact that the code designed from MT consisted of source and target pairs. Language Modeling meant that the input, output pairs are essentially a slice of sentence from the dataset. This simplified the tokenization and vocabulary processes as it is just for one language. Spacy was kept for tokenizing the English datasets. As much as possible, this code was faithful to the original machine translation implementation. One key difference from the original code was the dataset used. This is discussed in further deatil at the Dataset step.

## Imports

No changes were needed from the original assignment with the execption that the German Spacy tokenizer was no longer needed.

In [ ]:
!pip install portalocker
# Import the clear_output function from IPython.display module for clearing Jupyter Notebook cell output
from IPython.display import clear_output

# This command installs or upgrades the TorchData library, which provides utilities and abstractions for handling datasets in PyTorch, including data loading and preprocessing.
!pip install -U torchdata
# This command installs or upgrades the spaCy library, which is a powerful natural language processing (NLP) library in Python. SpaCy provides various functionalities such as tokenization, POS tagging, dependency parsing, and named entity recognition (NER).
!pip install -U spacy
# This command downloads the English language model 'en_core_web_sm' provided by spaCy. This model includes pre-trained word vectors and various linguistic annotations, making it suitable for tasks such as tokenization, part-of-speech tagging, and syntactic parsing.
!python -m spacy download en_core_web_sm

!pip install torchtext==0.15.1
# This command clears the output of the current cell in a Jupyter notebook environment. It can be useful for removing clutter and focusing on the most recent output or results.
clear_output()

In [ ]:
# Import the main PyTorch library
import torch

# Import the torch.nn module, which contains neural network-related classes and functions
import torch.nn as nn

# Import the torch.optim module, which provides optimization algorithms for updating neural network parameters
import torch.optim as optim

# Import the DataLoader class from torch.utils.data module, used for loading datasets in mini-batches during training
from torch.utils.data import Dataset, DataLoader

# Import the pad_sequence function from torch.nn.utils.rnn module, used for padding sequences within a batch
from torch.nn.utils.rnn import pad_sequence

# Import the get_tokenizer function from torchtext.data.utils module, used for tokenizing text data
from torchtext.data.utils import get_tokenizer

# Import the build_vocab_from_iterator function from torchtext.vocab module, used for building vocabulary from tokenized text data
from torchtext.vocab import build_vocab_from_iterator

# Import the Iterable and List classes/types from typing module for type hinting
from typing import Iterable, List

# Import the spaCy library for natural language processing tasks
import spacy

# Import the NumPy library with the alias np for numerical operations
import numpy as np

# Import the random module for generating random numbers
import random

# Import the math module for mathematical functions and constants
import math

# Import the time module for working with time
import time



In [ ]:
# Check if CUDA (GPU acceleration) is available on the system
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Execute the shell command to display GPU information using nvidia-smi
!nvidia-smi

Sat Mar 22 23:06:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

## Modeling

The following three cells contain the model codebase which remains unchanged (as requested) from the original assignment.

In [ ]:
# Since nn.Module has the backward function built-in, we can implicitly utilise it by inheriting nn.Module.
# eliminating the need to implement anything other than the forward pass.
class LSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        '''
          - Args:
                - input_size: The number of expected features in the input.
                - hidden_size: The number of features in the hidden state.
          - Functionality:
                - Initialises the LSTM layer with the specified input size, and hidden size.
                - Initialises weight and bias parameters.
        '''
        super(LSTMLayer, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Define parameters for a single LSTM layer
        # We mark the weight matrices as a parameter of the model using nn.Parameter
        # This is done to indicate that this tensor should be considered a model parameter.
        # So that backward function in pytorch consider these matrices during optimisation,
        # and to calculate the gradients with respect to them.
        self.W_f = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_f = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))

        self.W_i = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_i = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))

        self.W_g = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_g = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_g = nn.Parameter(torch.Tensor(hidden_size))

        self.W_o = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_o = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))

    def forward(self, input, h_prev, c_prev):
        # Concatenate input and previous hidden state
        # Forget gate
        # input: (batch_size, input_size) @ W_f: (input_size, hidden_size) -> (batch_size, hidden_size)
        # h_prev: (batch_size, hidden_size) @ U_f: (hidden_size, hidden_size) -> (batch_size, hidden_size)
        f = torch.sigmoid(input @ self.W_f + h_prev @ self.U_f + self.b_f)

        k = f * c_prev #mask


        # Input gate (Add gate)
        i = torch.sigmoid(input @ self.W_i + h_prev @ self.U_i + self.b_i)


        g = torch.tanh(input @ self.W_g + h_prev @ self.U_g + self.b_g) # Candidate cell state
        j = i * g #mask


        # Output gate
        o = torch.sigmoid(input @ self.W_o + h_prev @ self.U_o + self.b_o)

        # Update cell state
        c_next = k + j

        # Update hidden state
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

class StackLSTMLayers(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        '''
          - Args:
                - input_size: The number of expected features in the input.
                - hidden_size: The number of features in the hidden state.
                - num_layers: Number of Stacked recurrent layers.
          - Functionality:
                - Initialises the LSTM layer with the specified input size, hidden size, and number of layers.
                - Initialises weight and bias parameters for each layer.
        '''
        super(StackLSTMLayers, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Create stack of LSTM layers if num_layers > 1
        self.layers = nn.ModuleList([LSTMLayer(input_size if i == 0 else hidden_size, hidden_size) for i in range(num_layers)])

    def forward(self, input, hidden=None):
        if hidden is None:
            # initialise hidden and cell states if not provided
            hidden = self.init_hidden(input.size(1))

        # Unpack hidden states
        hiddens, cells = hidden

        outputs = []

        # Iterate through each time step
        for input_t in input:
            # Iterate through each layer
            for layer_idx, layer in enumerate(self.layers):
                # Pass input through the current layer
                hiddens[layer_idx], cells[layer_idx] = layer(input_t, hiddens[layer_idx], cells[layer_idx])

                # Update input for the next layer (if any)
                if layer_idx < self.num_layers - 1:
                    input_t = hiddens[layer_idx]

            # Append output of current time step
            outputs.append(hiddens[-1])

        # Stack outputs along the sequence dimension
        outputs = torch.stack(outputs, dim=0)


        # Return outputs, hidden states, and cell states
        return outputs, (hiddens, cells)

    def init_hidden(self, batch_size):
        # initialise hidden and cell states for each layer
        hiddens = [torch.zeros(batch_size, self.hidden_size, device=device) for _ in range(self.num_layers)]
        cells = [torch.zeros(batch_size, self.hidden_size, device=device) for _ in range(self.num_layers)]
        return hiddens, cells

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = StackLSTMLayers(emb_dim, hid_dim, n_layers)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        #embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):

        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = StackLSTMLayers(emb_dim, hid_dim, n_layers)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        Language modeling using an encoder-decoder architecture.

        src: [seq_len, batch_size]  -> Encoder input (full sentence)
        trg: [seq_len, batch_size]  -> Decoder input (shifted sequence starting with <sos>)
        teacher_forcing_ratio: Probability of using the true previous word instead of the predicted one
        """

        batch_size = trg.shape[1]
        seq_len = trg.shape[0]
        vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs
        outputs = torch.zeros(seq_len, batch_size, vocab_size).to(self.device)

        # Pass the input sequence through the encoder
        hidden, cell = self.encoder(src)

        # First input to the decoder is always the <sos> token from trg
        input = trg[0, :]

        # Loop over sequence length to predict each next word
        for t in range(1, seq_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output

            # Decide whether to use teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Get the most likely predicted word
            top1 = output.argmax(1)

            # Use ground truth next word (if teacher forcing), else use predicted word
            input = trg[t] if teacher_force else top1

        return outputs


##  Dataset

The biggest chnages to convert rfom MT to LM come with the dataset used (i.e. monolingual data vs source, target pairs) and the preprocessing of the data.

The text data was sourced from the Gutenberg Project which provides text files for many great literary works and can be sourced from the [Gutenberg Project Website](https://www.gutenberg.org/ebooks/31100) [1]. The works of author Jane Austen were chosen as there is numerous amounts of text data available to train, validate and test a model on.

In [ ]:
#file location - uploaded here to Google collab. This needs to be done to replicate or mount google drive.
DATA_DIR = "/content/"


# define custom dataset for this task
class JaneAustenDataset(Dataset):
  def __init__(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
      self.lines = f.readlines() #may create unwanted empty spaces

  def __len__(self):
    return len(self.lines)


  def __getitem__(self, idx):
    text = self.lines[idx].strip() #applied .strip to get rid of unwanted trailing whitespaces etc.
    return text

#loading files and creating dataset - split beforehand for convenience
train_dataset = JaneAustenDataset(DATA_DIR + "jane-austen-text-train.txt")
val_dataset = JaneAustenDataset(DATA_DIR + "jane-austen-text-val.txt")
test_dataset = JaneAustenDataset(DATA_DIR + "jane-austen-text-test.txt")

## Manual Initial Preprocessing##

The full text has nearly 4.5 million characters which was too large to run on Google Colab. Therefore the first two books, Persuasion and Northanger Abbey, were chosen and have just over 900,000 characters. For convenience, this was manually split into train, validate and test sets, with a 80\%, 10\% and 10\% split ratio.
Undesired text, such as Gutenberg Legal notes were manually removed. Including this text should be explored in the future as it may act as a generalization or regularization text as it very different in style to the works of Jane Austen. It should be noted that there may however, remain other notes in the final text. Examples of the removed text are included in the Appendix.

### Checking the dataset ###
Testing a few cases to check the data was loaded correctly:
*  train_dataset[0] -> 'PERSUASION'
*  train_dataset[10] -> 'Chapter 1'
*  train_dataset[100] -> 'influence had always been great, and they had gone on together most'

In [ ]:
train_dataset[0]

'PERSUASION'

## Tokenization & Vocabulary Creation
It was decided to keep the same tokenizer as given in the original, with the exception that now only a English Language tokenizer is needed. While keeping the processing as close to the original assignment, it was also desired to make the process more modular and adaptable [2]. The special characters were kept the same format as in the original assignemnt.

The vocabulary was optionally saved for future use.

In [ ]:
#load tokenizer from Spacy
nlp = spacy.load("en_core_web_sm")

#tokenize text using spacy tokenizer
def tokenizer(text):
  return [token.text for token in nlp(text)]


# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

#combine tokenization process and vocabulary creation by callilng tokenizer
def vocabulary(train_iter, vocab_path='vocab.pt', min_freq=2):
  """
  Tokenizes and creates a vocabulary from the training data. Saves Voacb for potential future use

  Args:
    train_iter(iterator): Iterates the training data
    vocab_path(str): optional path to save the vocabulary file for future use
    min_freq(int): optional parameter to exclude words below set threshold

  Returns:

  """
  #creating vocabulary
  vocab = build_vocab_from_iterator((tokenizer(text) for text in train_dataset), specials=special_symbols, min_freq=min_freq)


  #set deafult to unk
  vocab.set_default_index(UNK_IDX)

  #save the vocabulary - optional
  torch.save(vocab, vocab_path) #probably could drop this for this assignment

  return vocab

In [ ]:
vocab = vocabulary(train_dataset)

In [ ]:
len(vocab)

4430

#### Tokenizer and Collate Function
These funtions were kept largely the same, except that instead of source,target pairs from two lanaguges there is (input,output) pairs where the output is the next word of the input sentence.

In [ ]:
#convert text into tokens and add special characters
def tensor_transform(text):
  tokens = tokenizer(text)
  token_ids = [vocab[token] if token in vocab else UNK_IDX for token in tokens]
  #concatonates the sentence with the sos and eos tags in return
  return torch.cat((torch.tensor([BOS_IDX], dtype=torch.long),
                    torch.tensor(token_ids, dtype=torch.long),
                    torch.tensor([EOS_IDX], dtype=torch.long)))

In [ ]:
# function to collate data samples into batch tensors
def collate_fn(batch):
  input_batch, output_batch = [], []

  for sample in batch:
    token_ids = tensor_transform(sample.rstrip("\n")) #removes newlines at end

    input_batch.append(token_ids[:-1]) #w1,w2,w3,w4
    output_batch.append(token_ids[1:]) #w5

  #add padding term to create equal length sequences - moved outside the loop
  #length will deafult be longest sentence in batch
  input_batch = pad_sequence(input_batch, padding_value=PAD_IDX, batch_first = False) #batch_first set for LSTM dimensions
  output_batch = pad_sequence(output_batch, padding_value=PAD_IDX, batch_first=False)
  return input_batch, output_batch

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
valid_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

In [ ]:
#Testing Setup
sample_input, sample_output = next(iter(train_loader))

#Checking Tensor shapes
print("Input Shape:", sample_input.shape)  #(seq_len, batch_size)
print("Output batch shape:", sample_output.shape)  #(seq_len, batch_size)

# Debugging: Print first tokenized sequence
print("\nFirst input sequence (token IDs):", sample_input[:, 0])  # Checking first sentence
print("First output sequence (token IDs):", sample_output[:, 0])

Input Shape: torch.Size([23, 32])
Output batch shape: torch.Size([23, 32])

First input sequence (token IDs): tensor([2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
First output sequence (token IDs): tensor([0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
print("Total dataset size:", len(train_dataset))

Total dataset size: 12922


## Initializing Model
Initialization was kept mostly the same, however, there is now only one vocabulary now which simplifies the dimensions.

In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
BATCH_SIZE = 128

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,888,974 trainable parameters


## Train and Evaluate
This code is largely unchanged from the original assignment. The training takes approximately 30 minutes to run.

In [ ]:
# Example optimizer and criterion
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

def train(model, dataloader, optimizer, criterion, device, clip):

    model.train()

    epoch_loss = 0
    # iterating over dataloader batches
    for batch in dataloader:

        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        output = model(inputs, targets) #added targets here

        output = output.view(-1, output.size(-1)) #flatten for loss calculation
        targets = targets.view(-1).to(device)

        loss = criterion(output, targets)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) #

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):

    model.eval()

    epoch_loss = 0


    with torch.no_grad():

        for batch in dataloader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)


            output = model(inputs, targets, 0) #0 == turn off teacher forcing


            output = output.view(-1, output.size(-1))
            targets = targets.view(-1).to(device)


            loss = criterion(output, targets) #calculating loss

            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, device, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion, device)
    test_loss = evaluate(model, test_loader, criterion, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-seq2seq-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 2m 28s
	Train Loss: 5.902 | Train PPL: 365.928
	 Val. Loss: 5.888 |  Val. PPL: 360.659
	 Test. Loss: 5.890 |  Test. PPL: 361.546
Epoch: 02 | Time: 2m 26s
	Train Loss: 5.421 | Train PPL: 226.183
	 Val. Loss: 5.580 |  Val. PPL: 265.192
	 Test. Loss: 5.584 |  Test. PPL: 266.136
Epoch: 03 | Time: 2m 26s
	Train Loss: 5.106 | Train PPL: 164.969
	 Val. Loss: 5.439 |  Val. PPL: 230.231
	 Test. Loss: 5.448 |  Test. PPL: 232.286
Epoch: 04 | Time: 2m 25s
	Train Loss: 4.703 | Train PPL: 110.260
	 Val. Loss: 4.988 |  Val. PPL: 146.680
	 Test. Loss: 5.002 |  Test. PPL: 148.752
Epoch: 05 | Time: 2m 25s
	Train Loss: 4.344 | Train PPL:  76.984
	 Val. Loss: 4.738 |  Val. PPL: 114.186
	 Test. Loss: 4.757 |  Test. PPL: 116.345
Epoch: 06 | Time: 2m 26s
	Train Loss: 4.049 | Train PPL:  57.331
	 Val. Loss: 4.593 |  Val. PPL:  98.819
	 Test. Loss: 4.595 |  Test. PPL:  99.025
Epoch: 07 | Time: 2m 23s
	Train Loss: 3.793 | Train PPL:  44.407
	 Val. Loss: 4.465 |  Val. PPL:  86.964
	 Test. Loss:

##  Testing on 10 sentences

It was decided that the ten test sentences should be taken from a range of time to test how well the model could generalize.
The test sentences are mainly a range of famous quotes from famous books.
The quotes chosen are as follows:

*  Jane Austen, Pride and Prejudice
*  Jane Austen, Sense and Sensibility
*  Charles Dickens, A Tale of two Cities
*  Charlotte Brontë, Jane Eyre
*  Test Sentence given in Assignment

This gives a broad range of test sentences, some are in domain and should predict well such (1-5) as they are from the same author and should be similar in style. The sentences 6-9 are from authors just after Jane Austen's time and so therefore there may be stylistic similarities but also differences. The last sentence is a modern sentence and it is not expected to predict well on this sentence.

The last word was removed from each sentence to give the final test dataset. A generate function was added so that the model would choose the most likely next word of a given sentence based on its training. The results are included in the table below.

This [resource](https://github.com/bentrevett/pytorch-seq2seq/blob/main/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb) [3] which has a generation function for MT was adapted for this task.

**Note**: This notebook can either be run in full or the model path has been saved and shared and can be uploaded and loaded here using
```
model.load_state_dict(torch.load("lstm-seq2seq-model.pt"))
```

In [ ]:
def generate_next_word(model, sentence, vocab, device):
  model.eval() # set model to evaluation mode

  #Use tensor_transform which processes token for model
  input_tensor = tensor_transform(sentence).unsqueeze(1).to(device)
  #encoding
  hidden, cell = model.encoder(input_tensor)

  #decoding/generation
  input_word = input_tensor[-1, 0].unsqueeze(0) #i.e. start at last word of the given sentence
  with torch.no_grad():
    output, hidden, cell = model.decoder(input_word, hidden, cell)
    #get probability distribution for next word
    probs = torch.softmax(output, dim=1)
    #choose top word
    top_word = torch.argmax(probs)
    next_word = vocab.lookup_token(top_word.item())

  return next_word

In [ ]:
#testing ten sentences
test_sentences = ["It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a",
                     "Do anything rather than marry without",
                     "There are very few who have heart enough to be really in love without",
                     "The more I know of the world, the more I am convinced that I shall never see a man whom I can really love. I require so",
                     "Seven years would be insufficient to make some people acquainted with each other, and seven days are more than enough for",
                     "You have been the last dream of my",
                     "It was the best of times, it was the worst of",
                     "I am no bird; and no net ensnares me: I am a free human being with an independent",
                     "I would always rather be happy than",
                     "I love Neural"
]

for sentence in test_sentences:
  next_word = generate_next_word(model, sentence, vocab, device)
  print(f"Input: {sentence}")
  print(f"Predicted Next Word: {sentence} + {next_word}\n")

Input: It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a
Predicted Next Word: It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a + is

Input: Do anything rather than marry without
Predicted Next Word: Do anything rather than marry without + for

Input: There are very few who have heart enough to be really in love without
Predicted Next Word: There are very few who have heart enough to be really in love without + "

Input: The more I know of the world, the more I am convinced that I shall never see a man whom I can really love. I require so
Predicted Next Word: The more I know of the world, the more I am convinced that I shall never see a man whom I can really love. I require so + at

Input: Seven years would be insufficient to make some people acquainted with each other, and seven days are more than enough for
Predicted Next Word: Seven years would be insufficient 

## Results & Discussion

While the losses are still relatively high, it can be observed that with each epoch, the loss was dropping and that for most epochs, the Test Loss was the highest, followed by Validation Loss and then Train Loss, which is to be expected. The number of epochs was selected in order to avoid extremely long run times, especially with limited T4 GPU usage allowed by Google Colab.

It can be seen that these results are suboptimal, with none being accurately predicted.  Only sentence 9 exhibited 'local' coherency, with 'than to' being an old fashioned but grammatical piece of text.  For example, *I would rather be X than to be Y*. This dated construction makes sense given the historical style of writing on which the model was trained.
However, this is just locally coherent and none of the sentences make sense overall. This is likely due to the limited amount of text the model was trained on. As mentioned, the model path has been saved and is shared for further testing.


## Automatic Evaluation of Model

Automatic metrics, namely BLEU and BERTScore were implemented to evaluate the generated text.
Due to computation limitations, the evaluation was carried out in a separate notebook that has been shared along with this notebook called 'Evaluation of LM Text'.ipynb. For convenience the contents of that notebook are included below, with the code in markdown for future use.



```
# Installing required packages
!pip install nltk #to get sentence bleu
!pip install evaluate #to load in bertscore
!pip install bert_score #bertscore

# Imports needed
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
from  evaluate import load
bertscore = load("bertscore")
```
# Full Sentence evaluation
## Data
The data below consists of two lists:
* The actual famous quotes.
* The sentences whose last words were predicted using the model trained for the assignment.

We will now use to metrics to evaluate these sentences:
1.  BLEU Score [4] [5]
2.  BERT Score [6] [7]

```
# Real quotes
reference_sentences = [
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife",
    "Do anything rather than marry without affection",
    "There are very few who have heart enough to be really in love without encouragement",
    "The more I know of the world, the more I am convinced that I shall never see a man whom I can really love. I require so much",
    "Seven years would be insufficient to make some people acquainted with each other, and seven days are more than enough for others",
    "You have been the last dream of my soul",
    "It was the best of times, it was the worst of times",
    "I am no bird; and no net ensnares me: I am a free human being with an independent will",
    "I would always rather be happy than dignified",
    "I love Neural Networks"
]

#Text Generated by Model
generated_sentences = [
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a is",
    "Do anything rather than marry without for",
    "There are very few who have heart enough to be really in love without ”",
    "The more I know of the world, the more I am convinced that I shall never see a man whom I can really love. I require so at",
    "Seven years would be insufficient to make some people acquainted with each other, and seven days are more than enough for seemed",
    "You have been the last dream of my is",
    "It was the best of times, it was the worst of was",
    "I am no bird; and no net ensnares me: I am a free human being with an independent he",
    "I would always rather be happy than to",
    "I love Neural of"
]
```
## BLEU
BLEU is a popular evaluation metric for MT tasks and is used here to evaluate the next word prediction task.
BLEU compares the n-grams from human sentences to the translated (or in this case generated) sentences.
BLEU is implemented on each sentence below using python and this particular BLEU metric is between 0 (no matches) and 1 (all matched) [5].

```
#can assign weights to put more preferance on lower order n-grams.
#As BLEU can decrease with longer sentence
weights = (0.25, 0.25, 0.25, 0.75) #gave 4-grams a higher weight to match more of the sentence, i.e. including the generated word more often

#bleu requires the sentences to be split into words so the .split() was used. # .zip() creates (ref,gen) tuple
bleu_score = [sentence_bleu([reference.split()], generated.split(), weights=weights) for reference, generated in zip(reference_sentences, generated_sentences)]

#printing the score
for i, score in enumerate(bleu_score):
    print(f"Sentence {i+1} BLEU Score: {score:.2f}")

```

Results:
*  Sentence 1 BLEU Score: 0.93
*  Sentence 2 BLEU Score: 0.70
*  Sentence 3 BLEU Score: 0.89
*  Sentence 4 BLEU Score: 0.94
*  Sentence 5 BLEU Score: 0.93
*  Sentence 6 BLEU Score: 0.79
*  Sentence 7 BLEU Score: 0.85
*  Sentence 8 BLEU Score: 0.91
*  Sentence 9 BLEU Score: 0.75
*  Sentence 10 BLEU Score: 0.00


## BERTScore

BERTScore uses pre-trained contextual embeddings (BERT) and uses cosine similarity to match words in generated and reference sentences. The score is between 0 (no semantic similarity) and 1 (identical). [8]

```
#computing sentence level semantic similarity
bert_score = bertscore.compute(predictions=generated_sentences, references=reference_sentences, lang="en")

#printing results
for i in range(len(generated_sentences)):
  print(f"Sentence {i+1} BERTScore:")
  print(f"  Precision: {bert_score['precision'][i]:.2f}")
  print(f"  Recall: {bert_score['recall'][i]:.2f}")
  print(f"  F1: {bert_score['f1'][i]:.2f}")
```

Results:
*  Sentence 1 BERTScore:
  *  Precision: 0.96
  *  Recall: 0.96
  *  F1: 0.96
*  Sentence 2 BERTScore:
  *  Precision: 0.96
  *  Recall: 0.96
  *  F1: 0.96
*  Sentence 3 BERTScore:
  *  Precision: 0.96
  *  Recall: 0.97
  *  F1: 0.96
*  Sentence 4 BERTScore:
  *  Precision: 0.99
  *  Recall: 0.99
  *  F1: 0.99
*  Sentence 5 BERTScore:
  *  Precision: 0.97
  *  Recall: 0.98
  *  F1: 0.97
*  Sentence 6 BERTScore:
  *  Precision: 0.96
  *  Recall: 0.96
  *  F1: 0.96
*  Sentence 7 BERTScore:
  *  Precision: 0.96
  *  Recall: 0.97
  *  F1: 0.97
*  Sentence 8 BERTScore:
  *  Precision: 0.98
  *  Recall: 0.98
  *  F1: 0.98
*  Sentence 9 BERTScore:
  *  Precision: 0.97
  *  Recall: 0.95
  *  F1: 0.96
*  Sentence 10 BERTScore:
  *  Precision: 0.89
  *  Recall: 0.90
  *  F1: 0.89


## Sentence - Level Results:
As we can see, both BLEU and BERTScore return very high scores. The BLEU score was adjusted to have a higher weight towards 4-grams. As can be seen, this casused the BLEU score for the last sentence to be zero, which makes sense as the sentence has four words with one mismatch. However, this is due to the fact that the sentences are by the nature of the model, identical until the last word. To create a more accurate experiment, bigrams of the last two words from each sentence were taken and BLEU and BERTScore were obtained for these bigrams.

## Bigram study
```
# Real quotes
reference_bigrams = [
    "a wife",
    "without affection",
    "without encouragement",
    "so much",
    "for others",
    "my soul",
    "of times",
    "independent will",
    "than dignified",
    "Neural Networks"
]

#Text Generated by Model
generated_bigrams = [
    "a is",
    "without for",
    "without ”",
    "so at",
    "for seemed",
    "my is",
    "of was",
    "independent he",
    "than to",
    "Neural of"
]

#BLEU

weights_2 = (1,0,0,0)#i.e. only bigrams
bleu_bigrams = [sentence_bleu([reference.split()], generated.split(), weights=weights_2) for reference, generated in zip(reference_bigrams, generated_bigrams)]

#printing results
for i, score in enumerate(bleu_bigrams):
    print(f"Sentence {i+1} BLEU Score: {score:.2f}")
```

Results:
*  Sentence 1 BLEU Score: 0.50
*  Sentence 2 BLEU Score: 0.50
*  Sentence 3 BLEU Score: 0.50
*  Sentence 4 BLEU Score: 0.50
*  Sentence 5 BLEU Score: 0.50
*  Sentence 6 BLEU Score: 0.50
*  Sentence 7 BLEU Score: 0.50
*  Sentence 8 BLEU Score: 0.50
*  Sentence 9 BLEU Score: 0.50
*  Sentence 10 BLEU Score: 0.50

```
#BERTScore
bert_score_bigrams = bertscore.compute(predictions=generated_bigrams, references=reference_bigrams, lang="en")

#printing results
for i in range(len(generated_sentences)):
  print(f"Sentence {i+1} BERTScore:")
  print(f"  Precision: {bert_score_bigrams['precision'][i]:.2f}")
  print(f"  Recall: {bert_score_bigrams['recall'][i]:.2f}")
  print(f"  F1: {bert_score_bigrams['f1'][i]:.2f}")
```

Results:
*  Sentence 1 BERTScore:
  *  Precision: 0.88
  *  Recall: 0.88
  *  F1: 0.88
*  Sentence 2 BERTScore:
  *  Precision: 0.87
  *  Recall: 0.88
  *  F1: 0.87
*  Sentence 3 BERTScore:
  *  Precision: 0.80
  *  Recall: 0.86
  *  F1: 0.83
*  Sentence 4 BERTScore:
  *  Precision: 0.84
  *  Recall: 0.85
  *  F1: 0.85
*  Sentence 5 BERTScore:
  *  Precision: 0.84
  *  Recall: 0.87
  *  F1: 0.86
*  Sentence 6 BERTScore:
  *  Precision: 0.85
  *  Recall: 0.85
  *  F1: 0.85
*  Sentence 7 BERTScore:
  *  Precision: 0.87
  *  Recall: 0.86
  *  F1: 0.86
*  Sentence 8 BERTScore:
  *  Precision: 0.93
  *  Recall: 0.92
  *  F1: 0.92
*  Sentence 9 BERTScore:
  *  Precision: 0.88
  *  Recall: 0.85
  *  F1: 0.87
*  Sentence 10 BERTScore:
  *  Precision: 0.87
  *  Recall: 0.83
  *  F1: 0.85

## Results from Bigram Study
It can be seen here that the BLEU catches that none of the next words from the model were accurately predicted with a score of 0.5 for all. It is surprising that the BERTScore is as high as it is. However, it is likely that the more words generated by the model, the likely the BERTScore is to fall.


## Future Directions ##
As mentioned earlier, it was observed that while the Loss was still high in the final epoch, losses were gradually dropping. A future extension may be to increase the number of epochs further and examine whether this results in a improved performance. Another option could be to add more works of Jane Austen or other Authors. However, as previously mentioned, this may need to be done locally, due to Colab usage limitations. The model could be expanded to predicted more words than just the next and automatic metrics, such as BLEU and BERTScore may be suitable evaluation tools for this extended task.

# References
*  [1] Project Gutenberg, “The Complete Project Gutenberg Works of Jane Austen by Jane Austen,” Project Gutenberg, Jan. 25, 2010. Available: https://www.gutenberg.org/ebooks/31100. [Accessed: Mar. 29, 2025]
*  [2] Ebimsv, “GitHub - Ebimsv/Torch-Linguist: Language Modeling with PyTorch,” GitHub, 2023. Available: https://github.com/Ebimsv/Torch-Linguist. [Accessed: Mar. 29, 2025]
*  [3] bentrevett, “pytorch-seq2seq/1 - Sequence to Sequence Learning with Neural Networks.ipynb at main · bentrevett/pytorch-seq2seq,” GitHub, 2018. Available: https://github.com/bentrevett/pytorch-seq2seq/blob/main/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb. [Accessed: Mar. 29, 2025]
*  [4] K. Papineni, S. Roukos, T. Ward, and W.-J. Zhu, “BLEU: a Method for Automatic Evaluation of Machine Translation,” Proceedings of the 40th Annual Meeting on Association for Computational Linguistics  - ACL ’02, pp. 311–318, 2001, doi: https://doi.org/10.3115/1073083.1073135. Available: https://dl.acm.org/citation.cfm?id=1073135
*  [5] GeeksforGeeks, “NLP BLEU Score for Evaluating Neural Machine Translation Python,” GeeksforGeeks, Oct. 23, 2022. Available: https://www.geeksforgeeks.org/nlp-bleu-score-for-evaluating-neural-machine-translation-python/#what-is-bleu-score. [Accessed: Mar. 29, 2025]
*  [6] J. Devlin, M.-W. Chang, K. Lee, and K. Toutanova, “BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. ,” In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, vol. 1, pp. 4171–4186, 2019, Available: https://aclanthology.org/N19-1423/. [Accessed: Mar. 29, 2025]
*  [7] Tiiiger, “GitHub - Tiiiger/bert_score: BERT score for text generation,” GitHub, Feb. 20, 2023. Available: https://github.com/Tiiiger/bert_score?tab=readme-ov-file#usage. [Accessed: Mar. 29, 2025]
*  [8] “BERT Score - a Hugging Face Space by evaluate-metric,” huggingface.co. Available: https://huggingface.co/spaces/evaluate-metric/bertscore. [Accessed: Mar. 29, 2025]

## Appendix
Example of Text Manually Removed
````
The Project Gutenberg eBook of The Complete Project Gutenberg Works of Jane Austen
        
    This ebook is for the use of anyone anywhere in the United States and
    most other parts of the world at no cost and with almost no restrictions
    whatsoever. You may copy it, give it away or re-use it under the terms
    of the Project Gutenberg License included with this ebook or online
    at www.gutenberg.org. If you are not located in the United States,
    you will have to check the laws of the country where you are located
    before using this eBook.

    Title: The Complete Project Gutenberg Works of Jane Austen

    Author: Jane Austen

    Editor: David Widger

    Release date: January 25, 2010 [eBook \#31100]
                  Most recently updated: June 15, 2013

    Language: English

    Credits: Produced by David Widger


    *** START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE PROJECT GUTENBERG WORKS OF JANE AUSTEN ***

    Produced by many Project Gutenberg volunteers.


    THE WORKS OF JANE AUSTEN


    Edited by David Widger

    Project Gutenberg Editions



                DEDICATION

        This Jane Austen collection
            is dedicated to
        Alice Goodson [Hart] Woodby



    [Note: The accompanying HTML file has active links to all the volumes
    and chapters in this set.]


    CONTENTS:

      PERSUASION

      NORTHANGER ABBEY

      MANSFIELD PARK

      EMMA

      LADY SUSAN

      LOVE AND FREINDSHIP AND OTHER EARLY WORKS

      PRIDE AND PREJUDICE

      SENSE AND SENSIBILITY
````